In [2]:
import pandas as pd
import numpy as np

###  First will try without using column transformer how hectic is to trasnform the data (catergorical, numberical..etc)

In [3]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/campusx-official/100-days-of-machine-learning/main/day28-column-transformer/covid_toy.csv')

In [5]:
df.head(5)

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   age        100 non-null    int64  
 1   gender     100 non-null    object 
 2   fever      90 non-null     float64
 3   cough      100 non-null    object 
 4   city       100 non-null    object 
 5   has_covid  100 non-null    object 
dtypes: float64(1), int64(1), object(4)
memory usage: 4.8+ KB


#### we have to apply OneHot Encoding on gender and city column (bcz it contains nominal data) and Ordinal encpding on fever column bcz ot contains ordinal data. In fever column there is missing values need to fill those NaNs

In [7]:
df['city'].unique()

array(['Kolkata', 'Delhi', 'Mumbai', 'Bangalore'], dtype=object)

In [8]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain,ytest = train_test_split(df.drop(columns=['has_covid']),df['has_covid'],test_size=0.2)

xtrain.shape

(80, 5)

###### (now will transform without using column transform)

In [9]:
# simple imputer on fever column


si=SimpleImputer()

# si.fit(df[['fever']])
# si.transform(df[['fever']])  #instead of writing fit and transform sepately we can do it in single line using fit_transform

xtrain_fever=si.fit_transform(xtrain[['fever']])
Xtest_fever=si.fit_transform(xtest[['fever']])

xtrain_fever.shape

(80, 1)

In [10]:
# Ordinal Encoding

oe=OrdinalEncoder(categories=[['Mild','Strong']])

xtrain_cough=oe.fit_transform(xtrain[['cough']])
xtest_cough=oe.fit_transform(xtest[['cough']])

xtrain_cough.shape

(80, 1)

In [11]:
# OneHot Encoding

ohe=OneHotEncoder(drop='first',sparse=False)

xtrain_gc=ohe.fit_transform(xtrain[['gender','city']])
xtest_gc=ohe.fit_transform(xtest[['gender','city']])

xtrain_gc.shape

(80, 4)

In [12]:
#Extracting age column to join with other encoded columns

xtrain_age=xtrain.drop(columns=['gender','fever','cough','city']).values
xtest_age=xtest.drop(columns=['gender','fever','cough','city']).values

In [13]:
# join all the encoded columns
xtrain_transformed=np.concatenate((xtrain_age,xtrain_fever,xtrain_cough,xtrain_gc),axis=1)
xtest_transformed=np.concatenate((xtest_age,Xtest_fever,xtest_cough,xtest_gc),axis=1)

xtrain_transformed.shape

(80, 7)

## Now will use column transformed, which makes life easier

In [14]:
from sklearn.compose import ColumnTransformer

transformer=ColumnTransformer(transformers=[
    ('ts1',SimpleImputer(),['fever']),
    ('ts2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('ts3',OneHotEncoder(drop='first',sparse=False),['gender','city'])
],remainder='passthrough')

In [15]:
transformer.fit_transform(xtrain).shape

(80, 7)